<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#3-char-model" data-toc-modified-id="3-char-model-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>3 char model</a></span><ul class="toc-item"><li><span><a href="#Create-inputs" data-toc-modified-id="Create-inputs-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Create inputs</a></span></li><li><span><a href="#Create-and-train-model" data-toc-modified-id="Create-and-train-model-0.2.2"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Create and train model</a></span></li><li><span><a href="#Test-model" data-toc-modified-id="Test-model-0.2.3"><span class="toc-item-num">0.2.3&nbsp;&nbsp;</span>Test model</a></span></li></ul></li><li><span><a href="#Our-first-RNN!" data-toc-modified-id="Our-first-RNN!-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Our first RNN!</a></span><ul class="toc-item"><li><span><a href="#Create-inputs" data-toc-modified-id="Create-inputs-0.3.1"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Create inputs</a></span></li><li><span><a href="#Create-and-train-model" data-toc-modified-id="Create-and-train-model-0.3.2"><span class="toc-item-num">0.3.2&nbsp;&nbsp;</span>Create and train model</a></span></li><li><span><a href="#Test-model" data-toc-modified-id="Test-model-0.3.3"><span class="toc-item-num">0.3.3&nbsp;&nbsp;</span>Test model</a></span></li></ul></li><li><span><a href="#Our-first-RNN-with-keras!" data-toc-modified-id="Our-first-RNN-with-keras!-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Our first RNN with keras!</a></span></li><li><span><a href="#Returning-sequences" data-toc-modified-id="Returning-sequences-0.5"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Returning sequences</a></span><ul class="toc-item"><li><span><a href="#Create-inputs" data-toc-modified-id="Create-inputs-0.5.1"><span class="toc-item-num">0.5.1&nbsp;&nbsp;</span>Create inputs</a></span></li><li><span><a href="#Sequence-model-with-keras" data-toc-modified-id="Sequence-model-with-keras-0.5.2"><span class="toc-item-num">0.5.2&nbsp;&nbsp;</span>Sequence model with keras</a></span></li><li><span><a href="#One-hot-sequence-model-with-keras" data-toc-modified-id="One-hot-sequence-model-with-keras-0.5.3"><span class="toc-item-num">0.5.3&nbsp;&nbsp;</span>One-hot sequence model with keras</a></span></li></ul></li><li><span><a href="#Stateful-model-with-keras" data-toc-modified-id="Stateful-model-with-keras-0.6"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>Stateful model with keras</a></span></li><li><span><a href="#Pure-python-RNN!" data-toc-modified-id="Pure-python-RNN!-0.7"><span class="toc-item-num">0.7&nbsp;&nbsp;</span>Pure python RNN!</a></span><ul class="toc-item"><li><span><a href="#Set-up-basic-functions" data-toc-modified-id="Set-up-basic-functions-0.7.1"><span class="toc-item-num">0.7.1&nbsp;&nbsp;</span>Set up basic functions</a></span></li><li><span><a href="#Set-up-training" data-toc-modified-id="Set-up-training-0.7.2"><span class="toc-item-num">0.7.2&nbsp;&nbsp;</span>Set up training</a></span></li></ul></li><li><span><a href="#Keras-GRU" data-toc-modified-id="Keras-GRU-0.8"><span class="toc-item-num">0.8&nbsp;&nbsp;</span>Keras GRU</a></span><ul class="toc-item"><li><span><a href="#End" data-toc-modified-id="End-0.8.1"><span class="toc-item-num">0.8.1&nbsp;&nbsp;</span>End</a></span></li></ul></li></ul></li></ul></div>

In [1]:
%matplotlib inline
import utils
from utils import *
from __future__ import division, print_function

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


In [4]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [5]:
chars.insert(0, "\0")

In [6]:
print(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [7]:
print(chars[1:-6])

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']


In [8]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

Map from chars to indices and back again

In [9]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [10]:
idx = [char_indices[c] for c in text] # 把这个文章全部转成数字表示的列表

In [11]:
len(idx)

600893

In [12]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [13]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## 3 char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [ ]:
len(idx)

In [ ]:
a = 50

In [ ]:
step = 3

In [ ]:
for i in range(0,a-step-1,step):
    print(i,i+1,i+2,i+3)

In [ ]:
idx[:10]

In [ ]:
c1_dat = [idx[i] for i in range(0,len(idx)-step,step)]
c2_dat = [idx[i+1] for i in range(0,len(idx)-step,step)]
c3_dat = [idx[i+2] for i in range(0,len(idx)-step,step)]
c4_dat = [idx[i+3] for i in range(0,len(idx)-step,step)]

In [ ]:
c1_dat[:10]

In [ ]:
c1_dat[-10:]

我们的输入：

In [ ]:
x1 = np.stack(c1_dat[:-2]) # 为什么使用到-2的序列呢？
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

用x1，x2，x3组成的3个序列预测下一个单词序列x4

输出：

In [ ]:
y = np.stack(c4_dat[:-2])

对比一下原始idx：

In [ ]:
idx[:20]

看看看前10个输入和输出：

In [ ]:
x1[:10],x2[:10],x3[:10]

In [ ]:
y[:15]

这里就能看出来`step`变量的意思了，就是用3个字符去预测第四个字符：

```
序列：40 42 29 预测  30
序列：30 25 27 预测  29
序列：29  1  1 预测   1
...
...
```

In [ ]:
x1.shape,y.shape

The number of latent factors to create (i.e. the size of the embedding matrix)

嵌入矩阵的维度

In [ ]:
embedding_size = 42 #词向量，嵌入矩阵的维度

In [ ]:
def embedding_input(name,n_inputs,n_outputs):
    inputs = Input(shape=(1,),dtype='int64',name=name)
    emb = Embedding(input_dim=n_inputs,output_dim=n_outputs,input_length=1)(inputs)
    return inputs,Flatten()(emb)


Create inputs and embedding outputs for each of our 3 character inputs

作者认为嵌入矩阵比one-hot更好，所以这里第一个模型并没有使用Rnn，而是嵌入模型

In [ ]:
c1_in,c1 = embedding_input('c1',vocab_size,embedding_size)
c2_in,c2 = embedding_input('c2',vocab_size,embedding_size)
c3_in,c3 = embedding_input('c3',vocab_size,embedding_size)

### Create and train model

Pick a size for our hidden state

In [ ]:
n_hidden = 256

This is the 'green arrow' from our diagram - the layer operation from input to hidden.

In [ ]:
dense_in = Dense(n_hidden, activation='relu')

Our first hidden activation is simply this function applied to the result of the embedding of the first character.

In [ ]:
c1_hidden = dense_in(c1)

This is the 'orange arrow' from our diagram - the layer operation from hidden to hidden.

In [ ]:
dense_hidden = Dense(n_hidden, activation='tanh')

Our second and third hidden activations sum up the previous hidden state (after applying dense_hidden) to the new input state.

In [ ]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = keras.layers.add([c2_dense, hidden_2])

In [ ]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden =keras.layers.add([c3_dense, hidden_3])

This is the 'blue arrow' from our diagram - the layer operation from hidden to output.

In [ ]:
dense_out = Dense(vocab_size, activation='softmax') # 最终输出为所有字符可能性的概率，所以用softmax

The third hidden state is the input to our output layer.

In [ ]:
c4_out = dense_out(c3_hidden)

In [ ]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam()) # 没有one-hot就用这个

In [ ]:
model.optimizer.lr=0.000001

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=1)

In [ ]:
model.optimizer.lr=0.01

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

In [ ]:
model.optimizer.lr = 0.000001

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

In [ ]:
model.optimizer.lr = 0.01

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

### Test model

测试模型效果，就是用`steps`个的字符预测下一个

In [ ]:
[x1,x2,x3] # 用40 42 29 预测下一个

In [ ]:
def get_next(inputs):
    idxs = [char_indices[c] for c in inputs]
    arrs = [np.array(id)[np.newaxis] for id in idxs] # 添加新轴
    p = model.predict(arrs)
    i = np.argmax(p)
    print(i)
    return chars[i]

In [ ]:
get_next(' th')

In [ ]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [ ]:
get_next('phi')

In [ ]:
get_next(' th')

In [ ]:
get_next(' an')

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [14]:
steps=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.


这次是8个字符组成的列表

In [156]:
c_in_dat = [[idx[i+n] for i in range(0,len(idx)-steps,steps)] for n in range(steps)]

In [16]:
len(c_in_dat) # 8个列表的字符

8

建立每个列表的下一个字符，用连续的8个字符，预测下一个字符是什么

Then create a list of the next character in each of these series. This will be the labels for our model.

In [17]:
c_out_dat = [idx[i+steps] for i in range(0, len(idx)-steps,steps)]

In [18]:
len(c_out_dat)

75111

In [157]:
xs = [np.stack(c[:-2]) for c in c_in_dat] 

In [20]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [21]:
len(xs), xs[0].shape

(8, (75109,))

In [22]:
y = np.stack(c_out_dat[:-2])

In [23]:
y.shape

(75109,)

So each column below is one series of 8 characters from the text.

In [24]:
[xs[n][:steps] for n in range(steps)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

...and this is the next character after each sequence.

In [25]:
y[:steps]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [26]:
embedding_size = 42

### Create and train model

In [28]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [29]:
c_ins = [embedding_input('c'+str(n), vocab_size, embedding_size) for n in range(steps)]

In [30]:
n_hidden = 256

In [31]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', kernel_initializer='identity') # 生成单位矩阵的初始化器。
dense_out = Dense(vocab_size, activation='softmax')

In [32]:
c_ins[0][1]

<tf.Tensor 'flatten_1/Reshape:0' shape=(?, ?) dtype=float32>

The first character of each sequence goes through dense_in(), to create our first hidden activations.

In [33]:
hidden = dense_in(c_ins[0][1])

Then for each successive layer we combine the output of dense_in() on the next character with the output of dense_hidden() on the current hidden state, to create the new hidden state.

In [34]:
for i in range(1,steps):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = keras.layers.add([c_dense, hidden])

Putting the final hidden state through dense_out() gives us our output.

In [35]:
c_out = dense_out(hidden)

So now we can create our model.

In [36]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [41]:
model.fit(xs, y, batch_size=64, epochs=1)

Epoch 1/1
75109/75109 [==============================] - 6s 81us/step - loss: 2.5418


In [43]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

### Test model

In [44]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [45]:
get_next('for thos')

' '

In [46]:
get_next('part of ')

't'

In [47]:
get_next('queens a')

'n'

In [48]:
len(xs)

8

上面的代码很繁琐，现在用keras建立一个RNN：

## Our first RNN with keras!

In [49]:
n_neurons,n_embedding_size,n_steps,n_vocab_size = (256,42,8,86)

In [51]:
model = Sequential([
    Embedding(input_dim=n_vocab_size,output_dim=n_embedding_size,input_length=n_steps),
    SimpleRNN(n_hidden, activation='relu', recurrent_initializer='identity'),
    Dense(n_vocab_size,activation='softmax'),
])

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_4 (Dense)              (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [54]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [55]:
[xs[n][:steps] for n in range(steps)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [57]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [59]:
y

array([ 1, 33,  2, ..., 71, 61, 54])

In [61]:
# 产生8个序列，
xs = [[[c] for c in xs[n][:]] for n in range(steps)]

In [63]:
np.concatenate(xs,axis=1)

array([[40, 42, 29, ..., 27, 29,  1],
       [ 1,  1, 43, ..., 40, 39, 43],
       [33, 38, 31, ..., 61, 54, 73],
       ...,
       [72, 73, 62, ...,  2, 76, 68],
       [71, 65, 57, ..., 72,  2, 73],
       [61, 58,  2, ..., 57, 62, 56]])

In [64]:
np.concatenate(xs,axis=1).shape

(75109, 8)

In [62]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 7s 92us/step - loss: 2.7819
Epoch 2/8
75109/75109 [==============================] - 7s 89us/step - loss: 2.2742
Epoch 3/8
75109/75109 [==============================] - 7s 87us/step - loss: 2.0780
Epoch 4/8
75109/75109 [==============================] - 7s 87us/step - loss: 1.9329
Epoch 5/8
75109/75109 [==============================] - 7s 89us/step - loss: 1.8245
Epoch 6/8
75109/75109 [==============================] - 7s 90us/step - loss: 1.7435
Epoch 7/8
75109/75109 [==============================] - 7s 92us/step - loss: 1.6773
Epoch 8/8
75109/75109 [==============================] - 7s 92us/step - loss: 1.6233


In [65]:
def get_next_keras(inputs):
    idxs = [char_indices[c] for c in inputs]
    print(np.array(idxs).shape)
    arrs = np.array(idxs)[np.newaxis,:] # 增加一个维度,
    print(arrs)
    print(arrs.shape)
    p = model.predict(arrs)[0]
    print(p)
    return chars[np.argmax(p)]

In [66]:
get_next_keras('this is ')

(8,)
[[73 61 62 72  2 62 72  2]]
(1, 8)
[9.0714e-08 3.1111e-04 5.0563e-04 1.9398e-05 2.0867e-02 8.3466e-04 5.3731e-04 2.2798e-05 6.1139e-05
 5.5762e-04 3.6684e-05 8.1658e-07 1.0322e-05 1.0991e-05 3.1401e-06 1.8335e-06 1.9541e-06 1.3025e-06
 2.6157e-06 2.9315e-06 3.4288e-07 2.3152e-05 4.3754e-05 3.5023e-05 2.7245e-05 2.9415e-03 1.6380e-03
 3.0594e-03 1.6515e-03 6.3696e-03 9.9752e-04 7.6107e-03 8.0624e-04 7.0514e-03 1.0322e-03 7.9385e-04
 1.9673e-03 2.2728e-03 9.0751e-03 9.3241e-04 2.6652e-03 2.6579e-05 5.3168e-03 2.8048e-03 3.1768e-03
 3.5482e-04 1.5567e-04 2.6991e-03 4.6833e-06 1.4352e-04 1.3602e-05 9.9436e-04 6.7916e-07 5.9357e-05
 1.0756e-01 3.8513e-02 3.8449e-02 3.2212e-02 5.0399e-02 2.0459e-02 2.5512e-02 2.2620e-02 7.4995e-02
 6.7123e-03 5.6692e-03 1.5185e-02 4.9277e-02 4.3108e-02 3.8181e-02 2.4755e-02 4.8821e-03 3.7424e-02
 4.7396e-02 1.5535e-01 1.4468e-02 4.6842e-03 4.7294e-02 7.9333e-05 4.2211e-03 5.4424e-05 1.3861e-07
 4.1548e-07 1.7244e-07 2.7892e-07 1.1177e-06 3.3240e-07]


't'

In [67]:
get_next_keras('part of ')

(8,)
[[69 54 71 73  2 68 59  2]]
(1, 8)
[1.9207e-08 4.8747e-04 7.6812e-04 4.6288e-05 4.1522e-02 9.9254e-04 1.2319e-04 2.0472e-05 4.4410e-05
 8.6126e-04 3.8650e-05 4.2502e-08 1.4145e-06 1.2204e-06 2.1387e-06 4.7612e-07 7.5942e-08 2.0502e-07
 1.4160e-07 2.7693e-07 1.0131e-07 2.1116e-05 1.4616e-05 1.9114e-05 1.9458e-05 1.0801e-03 3.4671e-04
 3.8526e-04 2.7721e-03 8.3998e-03 1.2764e-03 7.1943e-03 2.9272e-03 6.6207e-04 1.8402e-03 1.4545e-03
 7.0137e-04 3.7418e-03 2.4094e-03 1.4227e-04 5.5118e-03 7.1431e-08 6.0277e-03 4.2759e-03 1.1892e-03
 2.2550e-04 3.5529e-05 5.3051e-04 2.7125e-06 3.9166e-05 2.9829e-07 1.4457e-04 1.5042e-06 2.1907e-05
 9.1390e-02 1.7559e-02 4.6532e-02 2.5621e-02 3.6824e-02 1.3248e-02 1.1740e-02 5.4457e-02 4.5748e-02
 6.3164e-03 8.1157e-03 2.5474e-02 7.3295e-02 1.2153e-02 1.8626e-02 4.4948e-02 3.9891e-03 1.5024e-02
 6.9560e-02 2.3379e-01 1.0397e-02 4.9019e-03 2.2724e-02 5.7249e-05 9.1436e-03 4.3465e-05 2.1806e-08
 2.3030e-08 6.7128e-08 5.1408e-08 1.8951e-07 6.4633e-08]


't'

In [68]:
get_next_keras('queens a')

(8,)
[[70 74 58 58 67 72  2 54]]
(1, 8)
[9.0091e-10 2.8429e-03 5.3851e-02 1.7063e-08 2.6400e-06 7.1213e-07 8.1123e-12 5.3089e-10 1.3042e-07
 4.3520e-09 2.3585e-04 7.4764e-07 2.0617e-06 8.7643e-08 2.8824e-06 1.1322e-06 2.2574e-07 6.0992e-07
 2.8082e-07 2.0866e-06 1.3941e-06 2.4420e-07 1.8600e-09 1.5476e-08 1.3958e-08 1.9769e-07 1.8332e-05
 2.6744e-06 1.3964e-06 7.0460e-08 2.7892e-05 1.1635e-05 1.4221e-08 3.7313e-06 8.4320e-13 2.6707e-06
 3.4993e-05 1.2994e-08 6.3110e-05 9.0694e-07 5.7858e-08 4.9139e-11 7.6998e-06 1.3102e-05 3.1399e-06
 1.8140e-06 1.5507e-05 1.3985e-05 1.2797e-05 1.8738e-12 1.4116e-09 5.1225e-09 3.8900e-08 1.4651e-12
 1.2379e-04 1.3905e-02 5.0368e-03 2.2269e-03 1.4096e-04 1.2510e-02 8.5946e-03 4.4270e-06 7.0981e-04
 3.4648e-06 5.1906e-05 3.7089e-02 5.7351e-03 6.7468e-01 8.3576e-05 1.4695e-02 2.0796e-05 3.1163e-02
 1.0774e-01 1.4798e-02 5.6186e-03 3.4952e-04 3.9951e-03 3.8325e-05 3.5111e-03 9.7378e-07 1.7178e-09
 1.0146e-10 2.2880e-09 2.1792e-09 1.3155e-09 1.2296e-09]


'n'

In [69]:
get_next_keras('I love y')

(8,)
[[33  2 65 68 75 58  2 78]]
(1, 8)
[4.6196e-07 7.1247e-04 4.3791e-03 6.2359e-05 1.1826e-04 1.7979e-04 1.2569e-08 7.8749e-07 2.0726e-05
 1.0167e-04 2.4447e-04 3.1435e-07 2.5934e-07 4.0885e-08 8.6074e-07 2.4257e-07 1.8328e-07 1.2178e-07
 1.0985e-07 1.7679e-07 2.5873e-07 5.9795e-06 1.6979e-06 1.1321e-05 2.1472e-06 8.7415e-04 5.1946e-04
 1.8193e-05 1.8084e-04 2.7363e-03 4.0846e-04 2.5552e-04 4.4935e-04 4.6046e-03 1.7808e-08 2.4211e-06
 5.2107e-06 3.1585e-07 4.2843e-04 2.1579e-03 2.5267e-06 5.6001e-07 2.0278e-04 3.5520e-04 8.8952e-04
 3.3730e-03 5.4139e-05 3.0978e-04 7.3315e-05 3.8524e-07 1.4705e-07 1.4375e-08 1.9107e-07 3.3870e-08
 3.9807e-02 3.1341e-03 8.2561e-03 7.9226e-04 1.0750e-01 2.1673e-02 1.4392e-03 8.1153e-03 1.4594e-02
 4.0172e-06 4.3063e-04 2.1055e-03 1.6535e-02 4.3682e-03 5.0206e-01 1.0708e-01 1.3031e-03 3.8648e-03
 2.1925e-03 5.8511e-02 2.3471e-02 8.9911e-03 2.5824e-02 3.1138e-03 1.1085e-02 1.6542e-06 1.1617e-07
 1.7866e-07 3.0232e-07 1.2193e-07 1.5063e-07 3.0447e-07]


'o'

## Returning sequences

### Create inputs

To use a sequence model, we can leave our input unchanged - but we have to change our output to a sequence (of course!)

Here, c_out_dat is identical to c_in_dat, but moved across 1 character.

In [70]:
steps

8

In [71]:
#c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
#            for n in range(cs)]

# c_out_dat = [idx[i+steps] for i in range(0, len(idx)-steps,steps)]
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-steps, steps)]  # 为什么？
            for n in range(steps)]

In [72]:
len(c_out_dat)

8

In [74]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

Reading down each column shows one set of inputs and outputs.

In [81]:
idx[:20]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1, 43, 45, 40, 40, 39, 43, 33, 38, 31, 2]

In [82]:
[xs[n][:steps] for n in range(steps)]

[[[40], [1], [33], [2], [72], [67], [73], [2]],
 [[42], [1], [38], [44], [2], [9], [61], [73]],
 [[29], [43], [31], [71], [54], [9], [58], [61]],
 [[30], [45], [2], [74], [2], [76], [67], [58]],
 [[25], [40], [73], [73], [76], [61], [24], [71]],
 [[27], [40], [61], [61], [68], [54], [2], [58]],
 [[29], [39], [54], [2], [66], [73], [33], [2]],
 [[1], [43], [73], [62], [54], [2], [72], [67]]]

In [83]:
[ys[n][:steps] for n in range(steps)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

### Sequence model with keras

In [167]:
n_neurons,n_embedding_size,n_steps,n_vocab_size

(256, 42, 8, 86)

To convert our previous keras model into a sequence model, simply add the 'return_sequences=True' parameter, and add TimeDistributed() around our dense layer.

In [96]:
model=Sequential([
        Embedding(vocab_size, n_embedding_size, input_length=n_steps),
        SimpleRNN(n_neurons, return_sequences=True, activation='relu', recurrent_initializer='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [97]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 8, 42)             3570      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 8, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 85)             21845     
Total params: 101,959
Trainable params: 101,959
Non-trainable params: 0
_________________________________________________________________


In [98]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [110]:
xs.shape

(75109, 8)

In [106]:
xs.shape

(75109, 8)

In [128]:
xs

array([[40, 42, 29, ..., 27, 29,  1],
       [ 1,  1, 43, ..., 40, 39, 43],
       [33, 38, 31, ..., 61, 54, 73],
       ...,
       [72, 73, 62, ...,  2, 76, 68],
       [71, 65, 57, ..., 72,  2, 73],
       [61, 58,  2, ..., 57, 62, 56]])

In [111]:
x_rnn=xs
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [112]:
x_rnn.shape, y_rnn.shape

((75109, 8), (75109, 8, 1))

In [113]:
model.fit(x_rnn, y_rnn, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 10s 134us/step - loss: 2.4334
Epoch 2/8
75109/75109 [==============================] - 10s 131us/step - loss: 2.0012
Epoch 3/8
75109/75109 [==============================] - 10s 135us/step - loss: 1.8854
Epoch 4/8
75109/75109 [==============================] - 10s 139us/step - loss: 1.8255
Epoch 5/8
75109/75109 [==============================] - 11s 143us/step - loss: 1.7863
Epoch 6/8
75109/75109 [==============================] - 11s 148us/step - loss: 1.7599
Epoch 7/8
75109/75109 [==============================] - 11s 151us/step - loss: 1.7393
Epoch 8/8
75109/75109 [==============================] - 11s 146us/step - loss: 1.7230


In [114]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [115]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 's', 'n', ' ']

### One-hot sequence model with keras

This is the keras version of the theano model that we're about to create.

In [168]:
vocab_size

85

In [169]:
n_vocab_size

86

In [170]:
model=Sequential([
        SimpleRNN(n_neurons, return_sequences=True, input_shape=(n_steps, vocab_size),
                  activation='relu', recurrent_initializer='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [162]:
c_in_dat = [[idx[i+n] for i in range(0,len(idx)-steps,steps)] for n in range(steps)]
xs = [np.stack(c[:-2]) for c in c_in_dat] 

In [122]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_y_rnn.shape

(75109, 8, 85)

In [164]:
oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

In [165]:
oh_x_rnn.shape

(75109, 8, 85)

In [171]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 10s 138us/step - loss: 2.4417
Epoch 2/8
75109/75109 [==============================] - 10s 134us/step - loss: 2.0368
Epoch 3/8
75109/75109 [==============================] - 11s 146us/step - loss: 1.9253
Epoch 4/8
75109/75109 [==============================] - 11s 149us/step - loss: 1.8603
Epoch 5/8
75109/75109 [==============================] - 11s 150us/step - loss: 1.8174
Epoch 6/8
75109/75109 [==============================] - 11s 151us/step - loss: 1.7858
Epoch 7/8
75109/75109 [==============================] - 11s 147us/step - loss: 1.7621
Epoch 8/8
75109/75109 [==============================] - 11s 145us/step - loss: 1.7434


In [172]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [173]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'd', 'n', ' ']

## Stateful model with keras

In [175]:
bs=64

A stateful model is easy to create (just add "stateful=True") but harder to train. We had to add batchnorm and use LSTM to get reasonable results.

When using stateful in keras, you have to also add 'batch_input_shape' to the first layer, and fix the batch size there.

In [176]:
model=Sequential([
        Embedding(vocab_size, n_embedding_size, input_length=n_steps, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_neurons, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [177]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [182]:
model.input_shape

(64, 8)

Since we're using a fixed batch shape, we have to ensure our inputs and outputs are a even multiple of the batch size.

In [178]:
mx = len(x_rnn)//bs*bs

In [179]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 25s 336us/step - loss: 2.2796
Epoch 2/4
75072/75072 [==============================] - 25s 329us/step - loss: 2.0206
Epoch 3/4
75072/75072 [==============================] - 25s 335us/step - loss: 1.9473
Epoch 4/4
75072/75072 [==============================] - 25s 339us/step - loss: 1.9054


In [180]:
model.optimizer.lr=1e-4

In [181]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 24s 316us/step - loss: 1.8774
Epoch 2/4
75072/75072 [==============================] - 23s 312us/step - loss: 1.8565
Epoch 3/4
75072/75072 [==============================] - 24s 314us/step - loss: 1.8402
Epoch 4/4
75072/75072 [==============================] - 24s 316us/step - loss: 1.8266


In [183]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 24s 317us/step - loss: 1.8151
Epoch 2/4
75072/75072 [==============================] - 25s 333us/step - loss: 1.8052
Epoch 3/4
75072/75072 [==============================] - 26s 351us/step - loss: 1.7963
Epoch 4/4
75072/75072 [==============================] - 24s 314us/step - loss: 1.7887


## Pure python RNN!

### Set up basic functions

Now we're going to try to repeat the above theano RNN, using just pure python (and numpy). Which means, we have to do everything ourselves, including defining the basic functions of a neural net! Below are all of the definitions, along with tests to check that they give the same answers as theano. The functions ending in `_d` are the derivatives of each function.

In [ ]:
def sigmoid(x): return 1/(1+np.exp(-x))
def sigmoid_d(x): 
    output = sigmoid(x)
    return output*(1-output)

In [ ]:
def relu(x): return np.maximum(0., x)
def relu_d(x): return (x > 0.)*1.

In [ ]:
relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

In [ ]:
def dist(a,b): return pow(a-b,2)
def dist_d(a,b): return 2*(a-b)

In [ ]:
import pdb

In [ ]:
eps = 1e-7
def x_entropy(pred, actual): 
    return -np.sum(actual * np.log(np.clip(pred, eps, 1-eps)))
def x_entropy_d(pred, actual): return -actual/pred

In [ ]:
def softmax(x): return np.exp(x)/np.exp(x).sum()

In [ ]:
def softmax_d(x):
    sm = softmax(x)
    res = np.expand_dims(-sm,-1)*sm
    res[np.diag_indices_from(res)] = sm*(1-sm)
    return res

In [ ]:
test_preds = np.array([0.2,0.7,0.1])
test_actuals = np.array([0.,1.,0.])
nnet.categorical_crossentropy(test_preds, test_actuals).eval()

In [ ]:
x_entropy(test_preds, test_actuals)

In [ ]:
test_inp = T.dvector()
test_out = nnet.categorical_crossentropy(test_inp, test_actuals)
test_grad = theano.function([test_inp], T.grad(test_out, test_inp))

In [ ]:
test_grad(test_preds)

In [ ]:
x_entropy_d(test_preds, test_actuals)

In [ ]:
pre_pred = random(oh_x_rnn[0][0].shape)
preds = softmax(pre_pred)
actual = oh_x_rnn[0][0]

In [ ]:
np.allclose(softmax_d(pre_pred).dot(x_entropy_d(preds,actual)), preds-actual)

In [ ]:
softmax(test_preds)

In [ ]:
nnet.softmax(test_preds).eval()

In [ ]:
test_out = T.flatten(nnet.softmax(test_inp))

In [ ]:
test_grad = theano.function([test_inp], theano.gradient.jacobian(test_out, test_inp))

In [ ]:
test_grad(test_preds)

In [ ]:
softmax_d(test_preds)

In [ ]:
act=relu
act_d = relu_d

In [ ]:
loss=x_entropy
loss_d=x_entropy_d

We also have to define our own scan function. Since we're not worrying about running things in parallel, it's very simple to implement:

In [ ]:
def scan(fn, start, seq):
    res = []
    prev = start
    for s in seq:
        app = fn(prev, s)
        res.append(app)
        prev = app
    return res

...for instance, `scan` on `+` is the cumulative sum.

In [ ]:
scan(lambda prev,curr: prev+curr, 0, range(5))

### Set up training

Let's now build the functions to do the forward and backward passes of our RNN. First, define our data and shape.

In [ ]:
inp = oh_x_rnn
outp = oh_y_rnn
n_input = vocab_size
n_output = vocab_size

In [ ]:
inp.shape, outp.shape

Here's the function to do a single forward pass of an RNN, for a single character.

In [ ]:
def one_char(prev, item):
    # Previous state
    tot_loss, pre_hidden, pre_pred, hidden, ypred = prev
    # Current inputs and output
    x, y = item
    pre_hidden = np.dot(x,w_x) + np.dot(hidden,w_h)
    hidden = act(pre_hidden)
    pre_pred = np.dot(hidden,w_y)
    ypred = softmax(pre_pred)
    return (
        # Keep track of loss so we can report it
        tot_loss+loss(ypred, y),
        # Used in backprop
        pre_hidden, pre_pred, 
        # Used in next iteration
        hidden, 
        # To provide predictions
        ypred)

We use `scan` to apply the above to a whole sequence of characters.

In [ ]:
def get_chars(n): return zip(inp[n], outp[n])
def one_fwd(n): return scan(one_char, (0,0,0,np.zeros(n_hidden),0), get_chars(n))

Now we can define the backward step. We use a loop to go through every element of the sequence. The derivatives are applying the chain rule to each step, and accumulating the gradients across the sequence.

In [ ]:
# "Columnify" a vector
def col(x): return x[:,newaxis]

def one_bkwd(args, n):
    global w_x,w_y,w_h

    i=inp[n]  # 8x86
    o=outp[n] # 8x86
    d_pre_hidden = np.zeros(n_hidden) # 256
    for p in reversed(range(len(i))):
        totloss, pre_hidden, pre_pred, hidden, ypred = args[p]
        x=i[p] # 86
        y=o[p] # 86
        d_pre_pred = softmax_d(pre_pred).dot(loss_d(ypred,y))  # 86
        d_pre_hidden = (np.dot(d_pre_hidden, w_h.T) 
                        + np.dot(d_pre_pred,w_y.T)) * act_d(pre_hidden) # 256

        # d(loss)/d(w_y) = d(loss)/d(pre_pred) * d(pre_pred)/d(w_y)
        w_y -= col(hidden) * d_pre_pred * alpha
        # d(loss)/d(w_h) = d(loss)/d(pre_hidden[p-1]) * d(pre_hidden[p-1])/d(w_h)
        if (p>0): w_h -= args[p-1][3].dot(d_pre_hidden) * alpha
        w_x -= col(x)*d_pre_hidden * alpha
    return d_pre_hidden

Now we can set up our initial weight matrices. Note that we're not using bias at all in this example, in order to keep things simpler.

In [ ]:
scale=math.sqrt(2./n_input)
w_x = normal(scale=scale, size=(n_input,n_hidden))
w_y = normal(scale=scale, size=(n_hidden, n_output))
w_h = np.eye(n_hidden, dtype=np.float32)

Our loop looks much like the theano loop in the previous section, except that we have to call the backwards step ourselves.

In [ ]:
overallError=0
alpha=0.0001
for n in range(10000):
    res = one_fwd(n)
    overallError+=res[-1][0]
    deriv = one_bkwd(res, n)
    if(n % 1000 == 999):
        print ("Error:{:.4f}; Gradient:{:.5f}".format(
                overallError/1000, np.linalg.norm(deriv)))
        overallError=0

## Keras GRU

Identical to the last keras rnn, but a GRU!

In [185]:
model=Sequential([
        GRU(n_neurons, return_sequences=True, input_shape=(n_steps, vocab_size),
                  activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [186]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/8
41664/75109 [===============>..............] - ETA: 9s - loss: 2.6079 

KeyboardInterrupt: 

In [ ]:
get_nexts_oh(' this is')

### End